## CLMs4BPO
Questo dimostratore genera una descrizione di una cabina di una imbarcazione di lusso a partire dai prodotti tecnologici presenti a bordo e dalle loro specifiche tecninche.

I dati di input devono seguire il seguente schema:

In [1]:
example_input="""
<product> Yamaha HTR-5660 6-Channel Digital Home Theater Receiver (Discontinued by Manufacturer)
<features> ['6.1-channel digital receiver with 85 watts per channel (x 6)', 'Decoding for DTS-ES Discrete 6.1, DTS-ES Matrix 6.1, DTS Neo:6, Dolby Digital EX, and Dolby Pro Logic II', '24 distinct sound programs with 44 variations meet every sound need', '4 optical and 1 coaxial digital inputs; 5 S-Video inputs and 2 outputs; 2 component-video inputs; set of front-panel A/V inputs', 'Measures 17-1/8 x 6-7/16 x 15-3/8 inches (W x H x D) and weighs 25 pounds']

<product> Sony KV-20FV12 20&quot; Trinitron Wega Flat-Screen TV
<features> ['20-inch TV with Trinitron flat picture tube; 22.25 x 18.38 x 19.88 inches (W x H x D)', '3-line digital comb filter improves image resolution and clarity', 'SpeedSurf rapid channel surfing, 5 favorite channel presets, and sleep timer', '2 sets of composite inputs (1 front and 1 rear), S-video input, audio output, and headphone jack', '10-watt DAC speaker system (5 watts x 2) with Matrix Surround and MTS decoder']

<product> GE AV93203 Cable Extension Adapter
<features> ['CableExtensionAdapters']

"""

/home/bagnol/miniconda3/envs/NLG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.02s/it]


Nella cella seguente si importa il file generate.py. Tramite questo script python il modello linguistico Alpaca viene caricato sulla GPU (se presente) o sulla CPU. Per questo passaggio è necessario di disporre di una RAM di almeno 14 Gb. 

Caricato il modello in memoria, la funzione generate_one_step() usa il modello per generare una descrizione a partire dai prodotti tecnologici presi come input.

In [2]:
from generate import generate_one_step

response=generate_one_step(example_input)
print(response)

This luxurious cruise ship cabin boasts a state-of-the-art home theatre setup featuring the latest technology from Yamaha. With six discrete channels, eighty five watts per channel, fourteen different sound profiles, two optical and one coaxial digital inputs, five S-Video inputs and two component video inputs, you can be sure to enjoy the best possible viewing experience on board. To top off the experience, the room also has a Sony KV-20FV12 20” Trinitron Wega Flat Screen TV which offers crystal clear images thanks to its three line digital comb filter and speedsurf feature. For added convenience, there is even a GE AV93203 Cable Extension Adapter so you never have to worry about running out of cables!
